Ou ka kòmande vèsyon enprime ak vèsyon elektwonik *Think Python 3e* soti nan
[Bookshop.org](https://bookshop.org/a/98697/9781098155438) ak
[Amazon](https://www.amazon.com/_/dp/1098155432?smid=ATVPDKIKX0DER&_encoding=UTF8&tag=oreilly20-20&_encoding=UTF8&tag=greenteapre01-20&linkCode=ur2&linkId=e2a529f94920295d27ec8a06e757dc7c&camp=1789&creative=9325).

In [ ]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + str(local))
    return filename

download('https://github.com/AllenDowney/ThinkPython/raw/v3/thinkpython.py');
download('https://github.com/AllenDowney/ThinkPython/raw/v3/diagram.py');

import thinkpython

# Analiz Tèks ak Generasyon

Apre sa, nou te kouvri estrikti done debaz Python yo — lis, diksyonè, ak tuples — epi kèk algoritm ki itilize yo.
Nan chapit sa a, nou pral itilize yo pou eksplore analiz tèks ak generasyon Markov:

* Analiz tèks se yon fason pou dekri relasyon estatistik ant mo nan yon dokiman, tankou pwobabilite ke yon mo swiv yon lòt, epi

* Generasyon Markov se yon fason pou pwodwi nouvo tèks ak mo ak fraz ki sanble ak tèks orijinal la.

Algoritm sa yo sanble ak kèk pati yon Modèl Langaj Gwo (LLM), ki se eleman kle yon chatbòt.

Nou pral kòmanse pa konte konbyen fwa chak mo parèt nan yon liv.
Apre sa nou pral gade pè mo, epi fè yon lis mo ki ka swiv chak mo.
Nou pral fè yon vèsyon senp yon générateur Markov, epi kòm egzèsis, ou pral gen chans fè yon vèsyon pi jeneral.

## Mo inik

Kòm premye etap pou analiz tèks, ann li yon liv — *The Strange Case Of Dr. Jekyll And Mr. Hyde* pa Robert Louis Stevenson — epi konte konbyen mo inik ki gen.
Enstriksyon pou telechaje liv la a se nan notebook chapit sa a.

Selil ki anba a telechaje liv la soti nan Project Gutenberg.

In [ ]:
download('https://www.gutenberg.org/cache/epub/43/pg43.txt');

Vèsyon Project Gutenberg lan gen enfòmasyon sou liv la an kòmansman ak enfòmasyon sou lisans lan anfen.
Nou pral itilize `clean_file` soti nan Chapit 8 pou retire materyèl sa a epi ekri yon fichye "clean" ki gen sèlman tèks liv la.

In [ ]:
def is_special_line(line):
    return line.strip().startswith('*** ')

def clean_file(input_file, output_file):
    reader = open(input_file, encoding='utf-8')
    writer = open(output_file, 'w')

    for line in reader:
        if is_special_line(line):
            break

    for line in reader:
        if is_special_line(line):
            break
        writer.write(line)
        
    reader.close()
    writer.close()

filename = 'dr_jekyll.txt'
clean_file('pg43.txt', filename)

Nou pral itilize yon bouk `for` pou li liy yo desde fichye a ak `split` pou separe liy yo an mo.
Apre sa, pou kenbe tras mo inik yo, nou pral sere chak mo kòm kle nan yon diksyonè.

In [ ]:
unique_words = {}
for line in open(filename):
    seq = line.split()
    for word in seq:
        unique_words[word] = 1

len(unique_words)

Longè diksyonè a se kantite mo inik — anviwon `6000` selon fason nou konte.
Men si nou enspekte yo, nou pral wè ke kèk pa mo valab.

Pa egzanp, ann gade mo ki pi long nan `unique_words`.
Nou ka itilize `sorted` pou triye mo yo, pase fonksyon `len` kòm agiman mo kle (keyword argument) pou mo yo triye pa longè.

In [ ]:
sorted(unique_words, key=len)[-5:]

Indèks `[-5:]` chwazi 5 dènye eleman lis triye a, ki se mo ki pi long yo.

Lis la gen mo lejitiman long, tankou "circumscription", ak kèk mo ak tirè, tankou "chocolate-coloured".
Men kèk mo ki pi long la se de mo separe pa yon tirè.
E gen lòt mo ki gen ponktiyasyon tankou pwen, pwen esklamasyon, ak mak sitasyon.

Donk, anvan nou kontinye, ann jere tirè ak lòt ponktiyasyon.

## Ponktiyasyon

Pou idantifye mo nan tèks la, nou dwe jere de pwoblèm:

* Lè gen yon tirè nan yon liy, nou ta dwe ranplase l ak yon espas — apre sa, lè nou itilize `split`, mo yo va separe.

* Apre nou separe mo yo, nou ka itilize `strip` pou retire ponktiyasyon.

Pou rezoud premye pwoblèm lan, nou ka itilize fonksyon sa a, ki pran yon chèn, ranplase tirè pa espas, separe chèn lan, epi retounen lis rezilta a.

In [ ]:
def split_line(line):
    return line.replace('—', ' ').split()

Remake `split_line` sèlman ranplase tirè, pa tirè debaz (hyphen).
Men yon egzanp.

In [ ]:
split_line('coolness—frightened')

Kounye a, pou retire ponktiyasyon depi kòmansman ak fen chak mo, nou ka itilize `strip`, men nou bezwen yon lis karaktè ki konsidere kòm ponktiyasyon.

Karaktè nan chèn Python yo an Unicode, ki se yon estanda entènasyonal pou repwezante lèt (preske) chak alfabè, nimewo, senbòl, siy ponktiyasyon, elatriye.
Modil `unicodedata` bay yon fonksyon `category` nou ka itilize pou konnen ki karaktè ki ponktiyasyon.
Lè nou bay li yon lèt, li retounen yon chèn ak enfòmasyon sou sa lèt la ye.

In [ ]:
import unicodedata
unicodedata.category('A')

Chèn kategori `'A'` se `'Lu'` — `'L'` vle di li se yon lèt ak `'u'` vle di li se majiskil.

Chèn kategori `'.'` se `'Po'` — `'P'` vle di li se ponktiyasyon ak `'o'` vle di sous-kategori "other".

In [ ]:
unicodedata.category('.')

Nou ka jwenn ponktiyasyon ki gen nan liv la pa tcheke karaktè ki gen kategori ki kòmanse ak `'P'`.
Bouk sa a sere mak ponktiyasyon inik yo nan yon diksyonè.

In [ ]:
punc_marks = {}
for line in open(filename):
    for char in line:
        category = unicodedata.category(char)
        if category.startswith('P'):
            punc_marks[char] = 1

Pou fè yon lis ponktiyasyon, nou ka reyini kle diksyonè a an yon chèn.

In [ ]:
punctuation = ''.join(punc_marks)
print(punctuation)

Kounye a nou konnen ki karaktè nan liv la ki ponktiyasyon, nou ka ekri yon fonksyon ki pran yon mo, retire ponktiyasyon soti nan kòmansman ak fen, epi konvèti li an miniskil.

In [ ]:
def clean_word(word):
    return word.strip(punctuation).lower()

Men yon egzanp.

In [ ]:
clean_word('“Behold!”')

Paske `strip` retire karaktè soti nan kòmansman ak fen, li pa touche mo ki gen tirè anndan, li kite yo menm jan an.

In [ ]:
clean_word('pocket-handkerchief')

Kounye a men yon bouk ki itilize `split_line` ak `clean_word` pou idantifye mo inik nan liv la.

In [ ]:
unique_words2 = {}
for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        unique_words2[word] = 1

len(unique_words2)

Ak definisyon sa a ki pi strik de sa ki yon mo ye, gen anviwon 4000 mo inik.
E nou ka konfime ke lis mo ki pi long la pi pwòp.

In [ ]:
sorted(unique_words2, key=len)[-5:]

Kounye a ann wè konbyen fwa chak mo itilize.

## Frekans mo

Bouk sa a kalkile frekans chak mo inik.

In [ ]:
word_counter = {}
for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        if word not in word_counter:
            word_counter[word] = 1
        else:
            word_counter[word] += 1

Premye fwa nou wè yon mo, nou inisyalize frekans li a `1`. Si nou wè menm mo ankò, nou inkremante frekans la.

Pou wè ki mo parèt pi souvan, nou ka itilize `items` pou jwenn pè kle-valè soti nan `word_counter`, epi triye yo pa dezyèm eleman pè a, ki se frekans la.
Nou pral defini yon fonksyon ki chwazi dezyèm eleman an.

In [ ]:
def second_element(t):
    return t[1]

Kounye a nou ka itilize `sorted` ak de paramèt mo kle:

* `key=second_element` vle di eleman yo ap triye selon frekans mo yo.

* `reverse=True` vle di eleman yo ap triye an lòd desandan, ak mo ki pi souvan parèt an premye.

In [ ]:
items = sorted(word_counter.items(), key=second_element, reverse=True)

Men senk mo ki pi souvan parèt.

In [ ]:
for word, freq in items[:5]:
    print(freq, word, sep='\t')

Nan seksyon pwochen, nou pral enkapsile bouk sa a an yon fonksyon.
E nou pral itilize li pou demontre yon nouvo karakteristik — paramèt opsyonèl.

## Paramèt opsyonèl

Nou te itilize fonksyon bati-an ki pran paramèt opsyonèl.
Pa egzanp, `round` pran yon paramèt opsyonèl `ndigits` ki endike konbyen chif desimal pou kenbe.

In [ ]:
round(3.141592653589793, ndigits=3)

Men se pa sèlman fonksyon bati-an — nou ka ekri fonksyon ak paramèt opsyonèl tou.
Pa egzanp, fonksyon sa a pran de paramèt, `word_counter` ak `num`.

In [ ]:
def print_most_common(word_counter, num=5):
    items = sorted(word_counter.items(), key=second_element, reverse=True)

    for word, freq in items[:num]:
        print(freq, word, sep='\t')

Dezyèm paramèt la sanble ak yon afektasyon, men se pa afektasyon — se yon paramèt opsyonèl.

Si ou rele fonksyon sa a ak yon agiman, `num` resevwa **valè defo** a, ki se `5`.

In [ ]:
print_most_common(word_counter)

Si ou rele fonksyon an ak de agiman, dezyèm agiman an asiyen a `num` olye de valè defo a.

In [ ]:
print_most_common(word_counter, 3)

Nan ka sa a, nou ta di agiman opsyonèl la **override** valè defo a.

Si yon fonksyon gen paramèt obligatwa ak opsyonèl, tout paramèt obligatwa yo dwe vini anvan, apre sa paramèt opsyonèl yo.

In [ ]:
%%expect SyntaxError

def bad_function(n=5, word_counter):
    return None

## Soustraksyon diksyonè

Imajine nou vle tcheke òtograf liv la — sa vle di, jwenn lis mo ki ka mal ekri.
Youn fason pou fè sa se jwenn mo nan liv la ki pa parèt nan yon lis mo valab.
Nan chapit anvan yo, nou te itilize yon lis mo ki konsidere valab pou jwèt mo tankou Scrabble.
Kounye a nou pral itilize lis sa a pou tcheke òtograf Robert Louis Stevenson.

Nou ka konsidere pwoblèm sa a kòm soustraksyon ansanbl — sa vle di, nou vle jwenn tout mo soti nan yon ansanbl (mo yo nan liv la) ki pa nan lòt la (mo yo nan lis la).

Selil sa a telechaje lis mo yo.

In [ ]:
download('https://raw.githubusercontent.com/AllenDowney/ThinkPython/v3/words.txt');

Menm jan anvan, nou ka li kontni `words.txt` ak separe li an yon lis chèn.

In [ ]:
word_list = open('words.txt').read().split()

Apre sa, nou pral sere mo yo kòm kle nan yon diksyonè pou nou ka itilize operatè `in` pou tcheke vit si yon mo valab.

In [ ]:
valid_words = {}
for word in word_list:
    valid_words[word] = 1

Kounye a, pou idantifye mo ki parèt nan liv la men pa nan lis mo a, nou pral itilize `subtract`, ki pran de diksyonè kòm paramèt epi retounen yon nouvo diksyonè ki gen tout kle soti nan premye a ki pa nan dezyèm lan.

In [ ]:
def subtract(d1, d2):
    res = {}
    for key in d1:
        if key not in d2:
            res[key] = d1[key]
    return res

In [ ]:
diff = subtract(word_counter, valid_words)

Pou jwenn kèk mo ki ka mal ekri, nou ka enprime mo ki pi komen nan `diff`.

In [ ]:
print_most_common(diff)

Mo ki pi komen "mal ekri" yo se pi fò non prop ak kèk mo yon sèl lèt (Mr. Utterson se zanmi ak avoka Dr. Jekyll).

Si nou chwazi mo ki parèt sèlman yon fwa, yo pi gen chans mo ki vrèman mal ekri.
Nou ka fè sa lè nou boukle sou eleman yo epi fè yon lis mo ki gen frekans `1`.

In [ ]:
singletons = []
for word, freq in diff.items():
    if freq == 1:
        singletons.append(word)

Men kèk dènye eleman nan lis la.

In [ ]:
singletons[-5:]

Pifò ladan yo se mo valab ki pa nan lis mo a.
Men `'reindue'` parèt kòm yon fòt `reinduce`, kidonk nou jwenn omwen yon erè lejitim.

## Nimewo o aza

Kòm yon etap pou generasyon tèks Markov, kounye a nou pral chwazi yon sekans mo o aza soti nan `word_counter`.
Men anvan ann pale de o aza.

Bay menm antre yo, pifò pwogram konpitè yo **detèministik**, sa vle di yo pwodui menm rezilta chak fwa.
Detèminis anjeneral bon bagay, paske nou atann menm kalkil bay menm rezilta.
Pou kèk aplikasyon, sepandan, nou vle pwogram lan enprezib.
Jwèt se youn egzanp, men gen plis.

Fè yon pwogram vrèman nondetèministik difisil, men gen fason pou imite li.
Youn se itilize algoritm ki pwodui nimewo **pseudorandom**.
Nimewo pseudorandom pa vrèman o aza paske yo pwodui pa yon kalkil detèministik, men jis pa gade nimewo yo, li difisil pou fè diferans ak o aza.

Modil `random` bay fonksyon ki pwodui nimewo pseudorandom — ki nou pral rele "o aza" pou fasilite.
Nou ka enpòte li konsa.

In [ ]:
import random

In [ ]:
# selil sa a inisyalize générateur nimewo o aza a pou li 
# pwodui menm sekans chak fwa notebook lan kouri.

random.seed(4)

Modil `random` bay yon fonksyon `choice` ki chwazi yon eleman soti nan yon lis o aza, ak menm pwobabilite pou chak eleman chwazi.

In [ ]:
t = [1, 2, 3]
random.choice(t)

Si ou rele fonksyon an ankò, ou ka jwenn menm eleman an ankò, oswa yon lòt eleman.

In [ ]:
random.choice(t)

Alafen, nou atann jwenn chak eleman apeprè menm kantite fwa.

Si ou itilize `choice` ak yon diksyonè, ou jwenn `KeyError`.

In [ ]:
%%expect KeyError

random.choice(word_counter)

Pou chwazi yon kle o aza, ou dwe mete kle yo nan yon lis epi apre sa rele `choice`.

In [ ]:
words = list(word_counter)
random.choice(words)

Si nou pwodui yon sekans mo o aza, li pa fè anpil sans.

In [ ]:
for i in range(6):
    word = random.choice(words)
    print(word, end=' ')

Pati pwoblèm lan se ke nou pa pran an kont ke gen mo ki pi komen pase lòt.
Rezilta yo ap pi bon si nou chwazi mo ak "weights" diferan, konsa kèk mo chwazi pi souvan pase lòt.

Si nou itilize valè `word_counter` kòm pwa, chak mo chwazi ak yon pwobabilite ki depann de frekans li.

In [ ]:
weights = word_counter.values()

Modil `random` bay yon lòt fonksyon `choices` ki pran pwa kòm paramèt opsyonèl.

In [ ]:
random.choices(words, weights=weights)

E li pran yon lòt paramèt opsyonèl, `k`, ki presize konbyen mo pou chwazi.

In [ ]:
random_words = random.choices(words, weights=weights, k=6)
random_words

Rezilta a se yon lis chèn nou ka reyini an yon bagay ki sanble plis ak yon fraz.

In [ ]:
' '.join(random_words)

Si ou chwazi mo soti nan liv la o aza, ou jwenn yon sans de vokabilè a, men yon seri mo o aza raman fè sans paske pa gen relasyon ant mo k ap vini youn apre lòt.
Pa egzanp, nan yon fraz reyèl ou atann yon atikil tankou "the" suiv pa yon adjectif oswa non, pa vre pa vèb oswa adverbe.
Donk pwochen etap la se gade relasyon ant mo ki vin youn apre lòt.

## Bigram

Olye nou gade yon mo a la fwa, kounye a nou pral gade sekans de mo, rele **bigram**.
Yon sekans twa mo rele **trigram**, epi yon sekans ki gen yon kantite mo pa presize rele **n-gram**.

Ann ekri yon pwogram ki jwenn tout bigram nan liv la ak konbyen fwa chak parèt.
Pou sere rezilta yo, nou pral itilize yon diksyonè kote

* Kle yo se tuples chèn ki reprezante bigram yo, ak 

* Valè yo se antye ki reprezante frekans.

Ann rele li `bigram_counter`.

In [ ]:
bigram_counter = {}

Fonksyon sa a pran yon lis de chèn kòm paramèt.
L ap fè yon tuple de de chèn sa yo, ki ka itilize kòm kle nan diksyonè.
Apre sa, l ap ajoute kle a nan `bigram_counter` si li pa egziste, oswa inkremante frekans si li egziste.

In [ ]:
def count_bigram(bigram):
    key = tuple(bigram)
    if key not in bigram_counter:
        bigram_counter[key] = 1
    else:
        bigram_counter[key] += 1

Pandan nou pase nan liv la, nou dwe kenbe tras chak pè mo ki vin youn apre lòt.
Kidonk, si nou wè sekans "man is not truly one", nou ta ajoute bigram "man is", "is not", "not truly", etsetera.

Pou kenbe tras bigram sa yo, nou pral itilize yon lis `window`, paske li sanble ak yon fenèt k ap glise sou paj liv la, ki montre sèlman de mo a la fwa.
Okòmansman, `window` vid.

In [ ]:
window = []

Nou pral itilize fonksyon sa a pou trete mo yo youn pa youn.

In [ ]:
def process_word(word):
    window.append(word)
    
    if len(window) == 2:
        count_bigram(window)
        window.pop(0)

Premye fwa fonksyon sa a rele, li ajoute mo a bay `window`.
Paske gen sèlman yon mo nan fenèt la, pa gen bigram ankò, fonksyon an fini.

Dezyèm fwa li rele — epi chak fwa apre sa — li ajoute yon dezyèm mo nan `window`.
Paske kounye a gen de mo, li rele `count_bigram` pou kenbe tras konbyen fwa chak bigram parèt.
Apre sa li itilize `pop` pou retire premye mo a depi fenèt la.

Pwogram ki anba a boukle sou mo nan liv la epi trete yo youn pa youn.

In [ ]:
for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        process_word(word)

Rezilta a se yon diksyonè ki map soti chak bigram jiska konbyen fwa li parèt.
Nou ka itilize `print_most_common` pou wè bigram ki pi komen yo.

In [ ]:
print_most_common(bigram_counter)

Gade rezilta yo, nou ka jwenn sans de ki pè mo ki pi gen chans parèt ansanm.
Nou ka tou itilize rezilta sa yo pou pwodui tèks o aza, konsa.

In [ ]:
random.seed(0)
bigrams = list(bigram_counter)
weights = bigram_counter.values()
random_bigrams = random.choices(bigrams, weights=weights, k=6)

`bigrams` se yon lis bigram ki parèt nan liv la.
`weights` se lis frekans yo, donk `random_bigrams` se yon echantiyon kote pwobabilite chwazi yon bigram pwopòsyònèl ak frekans li. 

Men rezilta yo.

In [ ]:
for pair in random_bigrams:
    print(' '.join(pair), end=' ')

Fason sa a pou pwodui tèks pi bon pase chwazi mo o aza, men li toujou pa fè anpil sans.

## Analiz Markov

Nou ka fè pi byen ak analiz tèks Markov, ki kalkile pou chak mo nan tèks la, lis mo ki ka swiv li.
Kòm egzanp, nou pral analize lirik sa yo soti nan chan Monty Python "Eric, the Half a Bee":

```
Half a bee, philosophically,
Must, ipso facto, half not be.
But half the bee has got to be
Vis a vis, its entity. D'you see?
```

Pou sere rezilta yo, nou pral itilize yon diksyonè ki map soti chak mo jiska lis mo ki swiv li.

In [ ]:
successor_map = {}

Kòm egzanp, ann kòmanse ak de premye mo yo nan chan an.

In [ ]:
first = 'half'
second = 'a'

Si premye mo a pa nan `successor_map`, nou dwe ajoute yon nouvo eleman ki map soti premye mo a jiska yon lis ki gen dezyèm mo a.

In [ ]:
successor_map[first] = [second]
successor_map

Si premye mo a deja nan diksyonè a, nou ka fouye li pou jwenn lis siksesè nou te wè jusquaprezan, epi ajoute nouvo a.

In [ ]:
first = 'half'
second = 'not'

successor_map[first].append(second)
successor_map

Fonksyon sa a encapsile etap sa yo.

In [ ]:
def add_bigram(bigram):
    first, second = bigram
    
    if first not in successor_map:
        successor_map[first] = [second]
    else:
        successor_map[first].append(second)

Si menm bigram parèt plis pase yon fwa, dezyèm mo a ajoute plis pase yon fwa nan lis la.
Konsa, `successor_map` kenbe tras konbyen fwa chak siksesè parèt.

Kòm nou te fè seksyon anvan, nou pral itilize yon lis `window` pou stoke pè mo konsekitif.
E nou pral itilize fonksyon sa a pou trete mo yo youn pa youn.

In [ ]:
def process_word_bigram(word):
    window.append(word)
    
    if len(window) == 2:
        add_bigram(window)
        window.pop(0)

Men ki jan nou itilize li pou trete mo yo nan chan an.

In [ ]:
successor_map = {}
window = []

song = """
Half a bee, philosophically,
Must, ipso facto, half not be.
But half the bee has got to be
Vis a vis, its entity. D'you see?
"""

for word in song.split():
    word = clean_word(word)
    process_word_bigram(word)

Men rezilta yo.

In [ ]:
successor_map

Mo `'half'` ka swiv pa `'a'`, `'not'`, oswa `'the'`.
Mo `'a'` ka swiv pa `'bee'` oswa `'vis'`.
Pifò lòt mo parèt sèlman yon fwa, kidonk yo swiv pa yon sèl mo.

Kounye a ann analize liv la.

In [ ]:
successor_map = {}
window = []

for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        process_word_bigram(word)

Nou ka fouye nenpòt mo epi jwenn mo ki ka swiv li.

In [ ]:
# M te itilize selil sa a pou jwenn yon predesesè ki gen bon kantite
# siksesè posib ak omwen yon mo ki repete.

def has_duplicates(t):
    return len(set(t)) < len(t)

for key, value in successor_map.items():
    if len(value) == 7 and has_duplicates(value):
        print(key, value)

In [ ]:
successor_map['going']

Nan lis siksesè sa a, remake mo `'to'` parèt twa fwa — lòt siksesè yo parèt sèlman yon fwa.

## Pwodui tèks

Nou ka itilize rezilta seksyon anvan an pou pwodui nouvo tèks ak menm relasyon ant mo konsekitif ak nan orijinal la.
Men ki jan li mache:

* Kòmanse ak nenpòt mo ki parèt nan tèks la, nou fouye lis siksesè posib li yo epi chwazi youn o aza.

* Apre sa, ak mo chwazi a, nou fouye lis siksesè posib li yo, epi chwazi youn o aza.

Nou ka repete pwosesis sa a pou pwodui konbyen mo nou vle.
Men kòm egzanp, ann kòmanse ak mo `'although'`.
Men mo ki ka swiv li.

In [ ]:
word = 'although'
successors = successor_map[word]
successors

In [ ]:
random.seed(2)

Nou ka itilize `choice` pou chwazi nan lis la ak pwobabilite egal.

In [ ]:
word = random.choice(successors)
word

Si menm mo a parèt plis pase yon fwa nan lis la, li gen plis chans chwazi.

Reprann etap sa yo, nou ka itilize bouk sa a pou pwodui yon sekans ki pi long.

In [ ]:
for i in range(10):
    successors = successor_map[word]
    word = random.choice(successors)
    print(word, end=' ')

Rezilta a sanble plis ak yon fraz, men li toujou pa fè anpil sans.

Nou ka fè pi byen si nou itilize plis pase yon mo kòm kle nan `successor_map`.
Pa egzanp, nou ka fè yon diksyonè ki map soti chak bigram — oswa trigram — jiska lis mo ki vini apre.
Kòm egzèsis, ou ap gen chans aplike analiz sa a epi wè ki rezilta li bay.

## Debògaj

Kounye a nou ap ekri pwogram pi gwo, epi ou ka remake ou pase plis tan ap debòge.
Si ou bloke sou yon erè difisil, men kèk bagay pou eseye:

* Li: Egzamine kòd ou, li l redi ou menm, epi tcheke si li di sa ou te vle di.

* Kouri: Eksperimante lè ou fè chanjman ak kouri vèsyon diferan. Souvan si ou montre bagay la kote li nesesè nan pwogram nan, pwoblèm lan vin evidan, men pafwa ou dwe konstwi eskafora.

* Reflechi: Pran tan panse! Ki kalite erè li ye: sentaks, execisyon,
    oswa semantik? Ki enfòmasyon ou ka jwenn soti nan mesaj erè yo,
    oswa soti nan pwodiksyon pwogram lan? Ki kalite erè ki te ka lakòz
    pwoblèm ou wè a? Kisa ou te chanje dènye a, anvan
    pwoblèm lan parèt?

* Rubberducking: Si ou eksplike pwoblèm lan bay yon lòt moun, pafwa ou jwenn
    repons lan anvan ou fini poze kesyon an. Souvan ou pa bezwen
    lòt moun; ou ka jis pale ak yon kannar kawotchou. Se konsa estrateji
    **rubber duck debugging** la rele. M pa envante sa — gade
    <https://en.wikipedia.org/wiki/Rubber_duck_debugging>.

* Retrete: A yon moman, pi bon bagay la se rekile — defè chanjman
    resan yo — jiskaske ou retounen sou yon pwogram k ap mache. Apre sa ou ka rikonstwi.
    
* Repoze: Si ou bay sèvo ou yon poz, pafwa li ap jwenn pwoblèm lan pou ou.

Kòmanse pwogramè pafwa ret blòke sou youn nan aktivite sa yo epi bliye lòt yo. Chak aktivite gen pwòp fason li ka echwe.

Pa egzanp, li kòd ou ap mache si pwoblèm lan se yon erè tipografik, men pa si pwoblèm lan se yon enkonpreyansyon konseptyèl.
Si ou pa konprann sa pwogram ou fè, ou ka li l 100 fwa epi pa janm wè erè a, paske erè a nan tèt ou.

Kouri eksperyans ka mache, espesyalman si ou kouri ti tès senp.
Men si ou kouri eksperyans san panse oswa li kòd ou, sa ka pran anpil tan pou w konprann sa k pase.

Ou dwe pran tan panse. Debòge sanble ak yon syans eksperimental. Ou ta dwe gen omwen yon ipotèz sou sa pwoblèm lan ye. Si gen de oswa plis posiblite, eseye panse a yon tès ki ta elimine youn.

Menm pi bon teknik debògaj yo ap echwe si gen twòp
erè, oswa si kòd ou ap eseye ranje a twò gwo ak konplike.
Pafwa pi bon opsyon an se retrete, senplifye pwogram lan jiskaske
ou retounen sou yon bagay ki mache.

Kòmanse pwogramè souvan ezite retrete paske yo pa
ka sipòte efase yon liy kòd (menmsi li mal). Si sa fè w
santi w miyò, kopye pwogram ou an yon lòt fichye anvan ou kòmanse
retire bagay. Apre sa ou ka kopye moso yo tounen youn pa youn.

Jwenn yon erè difisil mande li, kouri, reflechi, retrete, epi pafwa repoze.
Si ou bloke sou youn nan aktivite sa yo, eseye lòt yo.

## Glosè

**default value (valè defo):**
Valè asiyen a yon paramèt si pa gen agiman bay.

**override (override):**
 Ranplase valè defo ak yon agiman.

**deterministic (detèministik):**
 Yon pwogram detèministik fè menm bagay chak fwa li kouri, bay menm antre.

**pseudorandom (pseudorandom):**
 Yon sekans nimewo pseudorandom sanble o aza, men pwodui pa yon pwogram detèministik.

**bigram:**
Yon sekans de eleman, souvan mo.

**trigram:**
Yon sekans twa eleman.

**n-gram:**
Yon sekans ki gen yon kantite eleman pa detèmine.

**rubber duck debugging:**
Yon fason debòge lè ou eksplike pwoblèm lan fò aloud bay yon objè inanime.

## Egzèsis

In [ ]:
# Liy sa a di Jupyter pou bay enfòmasyon detaye sou debògaj
# lè gen yon erè execisyon. Kouri li anvan ou travay sou egzèsis yo.

%xmode Verbose

### Mande yon asistan vityèl

Nan `add_bigram`, enstriksyon `if` la kreye yon nouvo lis oswa ajoute yon eleman nan yon lis ki egziste deja, selon si kle a deja nan diksyonè a.

In [ ]:
def add_bigram(bigram):
    first, second = bigram
    
    if first not in successor_map:
        successor_map[first] = [second]
    else:
        successor_map[first].append(second)

Diksyonè bay yon metòd `setdefault` nou ka itilize pou fè menm bagay la pi kout.
Mande yon asistan vityèl ki jan li mache, oswa kopye `add_word` bay yon asistan vityèl epi mande "Ou ka reekri sa a itilize `setdefault`?"

Nan chapit sa a nou te aplike analiz tèks Markov ak generasyon.
Si ou curye, ou ka mande asistan vityèl plis enfòmasyon sou sijè a.
Youn nan bagay ou ka aprann se ke asistan vityèl itilize algoritm ki sanble anpil — men tou diferan sou pwen enpòtan.
Mande yon AV, "Ki diferans ki gen ant modèl langaj gwo tankou GPT ak analiz tèks Markov?"

### Egzèsis

Ekri yon fonksyon ki konte konbyen fwa chak trigram (sekans twa mo) parèt. 
Si ou teste fonksyon ou ak tèks *Dr. Jekyll and Mr. Hyde*, ou ta dwe jwenn ke trigram ki pi komen se "said the lawyer".

Sijesyon: Ekri yon fonksyon ki rele `count_trigram` ki sanble ak `count_bigram`. Apre sa ekri yon fonksyon `process_word_trigram` ki sanble ak `process_word_bigram`.

In [ ]:
# Solisyon pral mete la

In [ ]:
# Solisyon pral mete la

Ou ka itilize bouk sa a pou li liv la epi trete mo yo.

In [ ]:
trigram_counter = {}
window = []

for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        process_word_trigram(word)

Apre sa itilize `print_most_common` pou jwenn trigram ki pi komen yo nan liv la.

In [ ]:
print_most_common(trigram_counter)

### Egzèsis

Kounye a ann aplike analiz tèks Markov ak yon mapping ki soti nan chak bigram jiska lis siksesè posib.

Kòmanse ak `add_bigram`, ekri yon fonksyon ki rele `add_trigram` ki pran yon lis twa mo epi swa ajoute oswa mete ajou yon eleman nan `successor_map`, itilize de premye mo yo kòm kle ak twazyèm mo a kòm siksesè posib.

In [ ]:
# Solisyon pral mete la

Men yon vèsyon `process_word_trigram` ki rele `add_trigram`.

In [ ]:
def process_word_trigram(word):
    window.append(word)
    
    if len(window) == 3:
        add_trigram(window)
        window.pop(0)

Ou ka itilize bouk sa a pou teste fonksyon ou ak lirik "Eric, the Half a Bee".

In [ ]:
successor_map = {}
window = []

for string in song.split():
    word = string.strip(punctuation).lower()
    process_word_trigram(word)

Si fonksyon ou mache byen, predesesè `('half', 'a')` ta dwe map jiska yon lis ki gen yon sèl eleman `'bee'`.
An reyalite, jan sa rive, chak bigram nan chan sa a parèt sèlman yon fwa, kidonk tout valè yo nan `successor_map` gen yon sèl eleman.

In [ ]:
successor_map

Ou ka itilize bouk sa a pou teste fonksyon ou ak mo yo soti nan liv la.

In [ ]:
successor_map = {}
window = []

for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        process_word_trigram(word)

Nan egzèsis kap vini an, ou pral itilize rezilta yo pou pwodui nouvo tèks o aza.

### Egzèsis

Pou egzèsis sa a, nou pral sipoze `successor_map` se yon diksyonè ki map soti chak bigram jiska lis mo ki swiv li.

In [ ]:
# selil sa a inisyalize générateur nimewo o aza a pou li 
# kòmanse menm kote chak fwa notebook sa a kouri.

random.seed(3)

Pou pwodui tèks o aza, nou pral kòmanse pa chwazi yon kle o aza soti nan `successor_map`.

In [ ]:
successors = list(successor_map)
bigram = random.choice(successors)
bigram

Kounye a ekri yon bouk ki pwodui 50 mo an plis, swiv etap sa yo:

1. Nan `successor_map`, fouye lis mo ki ka swiv `bigram`.

2. Chwazi youn pa aza epi enprime li.

3. Pou pwochen iterasyon an, fè yon nouvo bigram ki gen dezyèm mo `bigram` la ak siksesè nou chwazi a.

Pa egzanp, si nou kòmanse ak bigram `('doubted', 'if')` epi chwazi `'from'` kòm siksesè li, pwochen bigram la se `('if', 'from')`.

In [ ]:
# Solisyon pral mete la

Si tout bagay mache, ou ta dwe jwenn ke tèks generate a sanble ak style orijinal la, epi kèk fraz fè sans, men tèks la ka pase soti nan yon sijè a yon lòt.

Kòm egzèsis an plis, modifye solisyon ou a de dènyè egzèsis yo pou itilize trigram kòm kle nan `successor_map`, epi gade ki efè sa a gen sou rezilta yo.